# Рекомендация тарифов

***Оглавление***

<a href='#part1'>***1. Описание проекта***</a>

<a href='#part2'>***2. Условные обозначения***</a>

<a href='#part3'>***3. Открытие файла и изучение информации***</a>

<a href='#part4'>***4. Разбитие данных на выборки***</a>

<a href='#part5'>***5. Исследование моделей***</a>

<a href='#part6'>***6. Проверка модели на тестовой выборке***</a>

<a href='#part7'>***7. Проверка модели на адекватность***</a>

<a id='part1'>***1. Описание проекта***</a>

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении данные о поведении клиентов, которые уже перешли на новые тарифы "смарт" и "ультра". Необходимо построить модель для задачи классификации, которая выберет подходящий тариф. 

Построим модель с максимально большим значением "accuracy", но не менее 0,75

<a id='part2'>***2. Условные обозначения***</a>

сalls — количество звонков

minutes — суммарная длительность звонков в минутах

messages — количество sms-сообщений

mb_used — израсходованный интернет-трафик в Мб

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

<a id='part3'>***3. Открытие файла и изучение информации***</a>

In [6]:
# загружаем библиотеки и открываем файл.
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


try:
    df=pd.read_csv('/datasets/users_behavior.csv')
except:
    df=pd.read_csv('C:/Users/ASUS/Desktop/практикум/модуль 2/спринт 1. Введение в машинное обучение/users_behavior.csv')

display(df.head())
# посчитаем кол-во каждого тарифа, чтобы оценить их соотношение в выборке
print(df['is_ultra'].value_counts())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


0    2229
1     985
Name: is_ultra, dtype: int64


<a id='part4'>***4. Разбитие данных на выборки***</a>

In [7]:
# т.к. соотношение тарифов в датасете не одинаковое, применим стратификацию
# разбиваем датасет сначала на обучающий и промежуточный, затем промежуточный 
# на тестовый и валидационный
df_train, df_1=train_test_split(df, random_state=1, test_size=0.40, shuffle=True, stratify=df['is_ultra'])
df_test, df_valid=train_test_split(df_1, random_state=1, test_size=0.50, shuffle=True, stratify=df_1['is_ultra'])


# проверяем размерности полученных датасетов
print(df_valid.shape)
print(df_test.shape)
print(df_train.shape)


# создаем признаки и цели для каждой из выборок
features_train=df_train.drop(['is_ultra'], axis=1)
target_train=df_train['is_ultra']

features_valid=df_valid.drop(['is_ultra'], axis=1)
target_valid=df_valid['is_ultra']

features_test=df_test.drop(['is_ultra'], axis=1)
target_test=df_test['is_ultra']

(643, 5)
(643, 5)
(1928, 5)


<a id='part5'>***5. Исследование моделей***</a>

In [8]:
# исследуем модель дерева
result_tree=0
tree_depth=0
for depth in range(1, 10):
    model_tree = DecisionTreeClassifier(random_state=1, max_depth=depth)
    
    model_tree.fit(features_train, target_train)
    predictions_valid = model_tree.predict(features_valid)
    tree_accuracy=accuracy_score(target_valid, predictions_valid)
    
    if tree_accuracy>result_tree:
        result_tree=tree_accuracy
        tree_depth=depth
print("Глубина дерева", tree_depth, ':', end='')
print(result_tree)
    
    
    
    
# исследуем модель случайного леса    
result_forest=0
estimators=0

for n in range(1, 11):
    model_forest=RandomForestClassifier(random_state=1, n_estimators=n)
    
    model_forest.fit(features_train, target_train)
#    result=model_forest.score(features_valid, target_valid)
    
    predictions_valid_forest=model_forest.predict(features_valid)
    result=accuracy_score(target_valid, predictions_valid_forest)
    if result>result_forest:
        result_forest=result
        estimators=n

print("Количество деревьев", n, ':', end='')
print(result_forest)  





# исследуем модель логистической регрессии
model_regression=LogisticRegression(random_state=1)
model_regression.fit(features_train, target_train)
result_regression=model_regression.score(features_valid, target_valid)
print(result_regression)

Глубина дерева 8 :0.8227060653188181
Количество деревьев 10 :0.8087091757387247
0.7573872472783826


### Модели дерева решений и случайного леса по качеству получаются одинаковыми, а логистическая регрессия хуже требуемой в задании. На следующем этапе проверим дерево решений на тестовой выборке.

<a id='part6'>***6. Проверка модели на тестовой выборке***</a> 

In [9]:
model_tree = DecisionTreeClassifier(random_state=1, max_depth=8)
model_tree.fit(features_train, target_train)
predictions_test = model_tree.predict(features_test)
tree_accuracy=accuracy_score(target_test, predictions_test)

print(tree_accuracy)

0.7916018662519441


### В целом, качество модели незначительно снизилось, но при этом осталось выше требуемого значения. 

<a id='part7'>***7. Проверка модели на адекватность***</a>

In [10]:
### для проверки модели на адекватность сравним ее со случайной моделью. 
### Тогда вероятность получить ответ "1" и ответ "0" будет 50% на 50%. 
### Количество ответов "1" и "0" также разделится пополам. 
### В этом случае accuracy=0,5*0,5+0,5*0,5= 0,25+0,25=0,5

### Вывод: полученная в результате расчетов модель значительно превосходит случайную (0,79 против 0,5), что говорит о ее адекватности.    

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
